In [165]:
import finiteAutomata as lfa
import csv

In [166]:
# this uses the ascii table of uppercase letters to create the states
def check(state):
    if state == 82:
        state += 2
    else:
        state += 1
    return state

def read_csv(file_path):
    try:
        with open(file_path, newline='') as csvfile:
            reader = csv.reader(csvfile)
            lines = [line for line in reader if line]
            print(lines)
        return lines
    except FileNotFoundError:
        print(f'Arquivo {file_path} não encontrado.')
        exit(1)

def create_states(lines):
    states = {}
    initialState = 0
    state = 64 
    states['S'] = {}

    for line in lines:
        if line[0] != "<":
            for char in line:
                if not initialState:
                    state = check(state)
                    initialState = 1
                    states['S'].setdefault(char, []).append(chr(state))
                    states[chr(state)] = {}
                else:
                    state = check(state)
                    if state == 84:
                        states[chr(state - 2)].setdefault(char, []).append(chr(state))
                    else:
                        states[chr(state - 1)].setdefault(char, []).append(chr(state))
                    states[chr(state)] = {}
            initialState = 0
        else:
            rule = line.replace(' ::= ', ' | ').split(' | ')
            input_state = 'S' if rule[0][1] == 'S' else chr(state)
            createState = 0

            for m in range(1, len(rule)):
                if rule[m][0] != 'ε':
                    if rule[0][1] != rule[m][2]:
                        state = check(state)
                        states[chr(state)] = {}
                        createState = 1
                    states[input_state].setdefault(rule[m][0], []).append(chr(state))
                else:
                    states[input_state].setdefault(None, []).append('ε')

            if createState == 1:
                state = check(state)
                states[chr(state)] = {}

    return states

In [167]:
import pandas as pd
 
df = pd.read_csv('./entrada.csv',  header=None)
df

,0
0,se
1,entao
2,senao
3,;
4,<S> ::= a<A> | e<A> | i<A> | o<A> | u<A>
5,<A> ::= a<A> | e<A> | i<A> | o<A> | u<A> | ε


In [168]:
from collections import OrderedDict
terminal_letters = []

for row in df.values:
    for char in row:
        for c in char:
            if c.islower() and c != 'ε':
                terminal_letters.append(c)

unique_letters = list(OrderedDict.fromkeys(terminal_letters))
unique_letters

['s', 'e', 'n', 't', 'a', 'o', 'i', 'u']

In [169]:
afnd = pd.DataFrame(columns=unique_letters)

afnd.insert(0, 'σ', check(64))
afnd['σ'] = afnd['σ'].apply(lambda x: create_states(df.values), axis=1)


In [170]:
afnd

,σ,s,e,n,t,a,o,i,u


In [171]:
lines = read_csv("./entrada.csv")
automaton_states = create_states(lines)

print("Autômato Finito Não-Determinístico:\n")
lfa.orderedStates(automaton_states)
lfa.determinize(automaton_states)
print("Autômato determinizado")
lfa.orderedStates(automaton_states)

[['se'], ['entao'], ['senao'], [';'], ['<S> ::= a<A> | e<A> | i<A> | o<A> | u<A>'], ['<A> ::= a<A> | e<A> | i<A> | o<A> | u<A> | Îµ']]
Autômato Finito Não-Determinístico:

A : {}
B : {}
C : {}
D : {}
E : {}
F : {}
S : {'se': ['A'], 'entao': ['B'], 'senao': ['C'], ';': ['D'], '<S> ::= a<A> | e<A> | i<A> | o<A> | u<A>': ['E'], '<A> ::= a<A> | e<A> | i<A> | o<A> | u<A> | Îµ': ['F']}
Autômato determinizado
A : {}
B : {}
C : {}
D : {}
E : {}
F : {}
S : {'se': ['A'], 'entao': ['B'], 'senao': ['C'], ';': ['D'], '<S> ::= a<A> | e<A> | i<A> | o<A> | u<A>': ['E'], '<A> ::= a<A> | e<A> | i<A> | o<A> | u<A> | Îµ': ['F']}
